<a href="https://colab.research.google.com/github/airoprojects/visiope/blob/main/dataloader/load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import torch 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [15]:
import gdown
from pathlib import Path

# set up dataset directory path
ckpt = Path('/content/drive/MyDrive/Dataset/')
ckpt.mkdir(exist_ok=True, parents=True)

#https://drive.google.com/file/d/1eW9Ah9DDEY02CTHCrRYLGPmiGZvCTKK4/view?usp=share_link
# get url of zipped dataset
url = 'https://drive.google.com/uc?id=1eW9Ah9DDEY02CTHCrRYLGPmiGZvCTKK4'

# set up zip download location and start download
output = '/content/ai4mars-dataset-merged-0.1.zip'
gdown.download(url, output, quiet=False)

# unzip dataset
#!unzip "/content/ai4mars-dataset-merged-0.1.zip" -d "/content/drive/MyDrive/Dataset/"

Downloading...
From: https://drive.google.com/uc?id=1eW9Ah9DDEY02CTHCrRYLGPmiGZvCTKK4
To: /content/ai4mars-dataset-merged-0.1.zip
100%|██████████| 6.15G/6.15G [00:26<00:00, 235MB/s]


'/content/ai4mars-dataset-merged-0.1.zip'

In [ ]:
# unzip dataset
!unzip "/content/ai4mars-dataset-merged-0.1.zip" -d "/content/drive/MyDrive/Dataset/"

Archive:  /content/ai4mars-dataset-merged-0.1.zip
   creating: /content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1/
  inflating: /content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1/info.txt  
   creating: /content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1/mer/
 extracting: /content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1/mer/WIP  
   creating: /content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1/msl/
   creating: /content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1/msl/images/
   creating: /content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1/msl/images/edr/
  inflating: /content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1/msl/images/edr/current.txt  
  inflating: /content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1/msl/images/edr/edr_remain.txt  
  inflating: /content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1/msl/images/edr/get_edr.txt  
  inflating: /content/drive/MyDrive/Dataset/ai4mars-dataset-merged-0.1/msl/images/edr/NLA_

In [ ]:
#Path
images = "ai4mars-dataset-merged-0.1/msl/images"
label_train = "ai4mars-dataset-merged-0.1/msl/labels/train"
label_test_1ag = "ai4mars-dataset-merged-0.1/msl/labels/test/masked-gold-min1-100agree"
label_test_2ag = "ai4mars-dataset-merged-0.1/msl/labels/test/masked-gold-min2-100agree"
label_test_3ag = "ai4mars-dataset-merged-0.1/msl/labels/test/masked-gold-min3-100agree"
edr = images + "/edr"
mxy = images + "/mxy" 
rng = images + "/rng-30m"


edr_files = os.listdir(edr)
label_train_files = os.listdir(label_train)
label_test_files_1 = os.listdir(label_test_1ag)
label_test_files_2 = os.listdir(label_test_2ag)
label_test_files_3 = os.listdir(label_test_3ag)



X = []
y = []

y1 = []
y2 = []
y3 = []

c = 0
size = 224



for label in label_test_files_1:
    path_label = os.path.join(label_test_1ag, label)

    img_arr = cv2.imread(path_label,0) #read image
    img_arr = cv2.resize(img_arr, dsize=(size, size),interpolation=cv2.INTER_NEAREST) #resize
    y1.append(img_arr)

for label in label_test_files_2:
    path_label = os.path.join(label_test_1ag, label)

    img_arr = cv2.imread(path_label,0) #read image
    img_arr = cv2.resize(img_arr, dsize=(size, size),interpolation=cv2.INTER_NEAREST) #resize
    y2.append(img_arr)

for label in label_test_files_3:
    path_label = os.path.join(label_test_1ag, label)

    img_arr = cv2.imread(path_label,0) #read image
    img_arr = cv2.resize(img_arr, dsize=(size, size),interpolation=cv2.INTER_NEAREST) #resize
    y3.append(img_arr)
    

    


for label in label_train_files:


 
    img_name = label[:-4] + ".JPG" #Names of images match names of labels, except for the extension (JPG, png)

    if img_name in edr_files:

        img_path = os.path.join(edr, img_name) #get the full path
        
        # this is ugly, to change!
        img_arr = cv2.imread(img_path) #read image
        img_arr = cv2.resize(img_arr, dsize=(size, size)) #resize


        label_path = os.path.join(label_train, label)
        lab_arr = cv2.imread(label_path,0) #0 mean read as greyscale image
        lab_arr = cv2.resize(lab_arr, (size,size), interpolation = cv2.INTER_NEAREST)

        X.append(img_arr)
        print(label_path)
        y.append(lab_arr)

        #this control how much images you want
        c+=1
        if c==200: break


X = np.asanyarray(X, dtype= np.float32) / 255
y = np.array(y, dtype= np.int64)







# 0 - soil --> 0 
# 1 - bedrock --> 1
# 2 - sand --> 2
# 3 - big rock --> 3
# 255 -> 4 - NULL (no label)


y[y==255] = 4

print(X.shape, y.shape)
print(y[1])


#from numpy array to torch

Xt = torch.from_numpy(X)
yt = torch.from_numpy(y)

print(Xt)




'''
plt.imshow(X[0])
plt.show()
'''



'''
validation set parametrized to decide if you want only 1, 2, oe togheter
parametrized 
'''


In [6]:
# Trying to share data between notebooks
np.save('X.npy',X)
np.save('y.npy',y)